In [1]:
using CobraTools
using JuMP
using Gurobi # use your favourite solver
using Measurements
using LinearAlgebra
using JLD
using Plots
pyplot()

┌ Info: Precompiling CobraTools [16a0023f-0a72-4835-9977-a2aa9bff6e75]
└ @ Base loading.jl:1278


Plots.PyPlotBackend()

### Import model

In [2]:
# E. coli model
modelpath = joinpath("..", "models", "iJO1366.json") 
model = CobraTools.read_model(modelpath)
# gibbs = CobraTools.mapGibbs(model.rxns) # very slow - rather just import this - will need to reload for other models

## issue
decomp = JLD.load(joinpath("..","data", "dgzeros.jld"), "gibbs")
gibbs = Dict{String, Measurement{Float64}}()
for (k, vs) in decomp
    gibbs[k] = vs[1] ± vs[2]
end
##

ecoli_kJmolCarbon = -37.36 ± 8.55 # formation of biomass kJ/mol 74.36 ± 8.67
model

Constraint based model: iJO1366
Number of reactions: 2583
Number of metabolites: 1805
Number of genes: 1367


In [3]:
cbmodel, v, mb, ubs, lbs = CobraTools.CBM(model);
set_optimizer(cbmodel, Gurobi.Optimizer)
set_optimizer_attribute(cbmodel, "OutputFlag", 0) # quiet

biomass_index = model[findfirst(model.rxns, "BIOMASS_Ec_iJO1366_WT_53p95M")] 
glucose_index = model[findfirst(model.rxns, "EX_glc__D_e")]
o2_index = model[findfirst(model.rxns, "EX_o2_e")]
atpm_index = model[findfirst(model.rxns, "ATPM")]

Academic license - for non-commercial use only - expires 2021-03-27


716

In [4]:
# Fix glucose use 1.0 then normalization is easy. NB - if not 1 then change normalization!!
CobraTools.set_bound(glucose_index, ubs, lbs; ub=-1.0, lb=-1.0)

# Aerobic
# CobraTools.set_bound(o2_index, ubs, lbs; ub=1000.0, lb=-1000.0)
# Anaerobic
CobraTools.set_bound(o2_index, ubs, lbs; ub=1000.0, lb=0.0)

# No free ATP generation
CobraTools.set_bound(atpm_index, ubs, lbs; ub=1000.0, lb=0.0)

@objective(cbmodel, Max, v[biomass_index])

optimize!(cbmodel) 
termination_status(cbmodel) != MOI.OPTIMAL && @warn "Optimization issue..."

μ = objective_value(cbmodel)

0.027710098044192628

In [5]:
### Fix biomass as a constraint
CobraTools.set_bound(biomass_index, ubs, lbs; ub=μ, lb=0.99*μ)  

In [6]:
wpoints = CobraTools.get_warmup_points(cbmodel, v, ubs, lbs) # very slow

2583×5144 Array{Float64,2}:
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0           0.0              0.0           0.0
 -0.00953335   -0.00953335   -0.00943802      -0.00176154   -0.0101388
 -6.65042e-7   -6.65042e-7   -6.58392e-7      -6.58392e-7   -6.58392e-7
  6.17935e-6    6.17935e-6    6.11756e-6       6.11756e-6    6.11756e-6
  3.1063e-5     3.1063e-5     3.07524e-5   …   3.01489e-5    3.01489e-5
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0             -6.03526e-7   -6.03526e-7
  3.70761e-5    3.70761e-5    3.67054e-5       3.61018e-5    3.61018e-5
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0          …   0.0           0.0
 -1.0          -1.0          -1.0             -1.0          -1.0
  0.0           0.0           0.0              0.0           0.0
  ⋮                                        ⋱                
  0.0           0.0      

In [9]:
Nsamples = 100_000
samples = CobraTools.achr(Nsamples, wpoints, model, ubs, lbs) 

┌ Warning: Error:  no feasible direction found.
└ @ CobraTools C:\Users\St. Elmo\Dropbox\PDResearch\projects\CobraTools\src\sampling.jl:139


2583×1001 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [ ]:
ΔG_exts = Measurement{Float64}[]
for s in 900_000:1000:Nsamples
    fluxes = CobraTools.map_fluxes(samples[:, s], model)
    carbon_ex = CobraTools.atom_exchange(fluxes, model)["C"] # carbon flux
    ΔG_ext, missing_ext =  CobraTools.map_gibbs_external(fluxes, gibbs) 
    ΔG_ext -= carbon_ex*ecoli_kJmolCarbon # minus because carbons consumed
    push!(ΔG_exts, ΔG_ext)
end

In [ ]:
plot(ΔG_exts)

In [ ]:
# plot(μs./μ_max, ΔG_exts, label="External", ylabel="ΔG [kJ/mol Glc]", xlabel="Anabolic flux fraction")
# plot!(μs./μ_max, ΔG_ints, label="Internal", ylabel="ΔG [kJ/mol Glc]", xlabel="Anabolic flux fraction")
# savefig("aerobic_dg.png")